In [1]:
from src import loader
from riix.models.elo import Elo
from riix.models.glicko2 import Glicko2
from riix.models.ttt import TrueSkillThroughTime
from riix.utils import MatchupDataset, split_matchup_dataset, generate_matchup_data
from riix.metrics import binary_metrics_suite
#from riix.models.eloplus import EloPlus
from riix.models.glicko import Glicko
import polars as pl
import numpy as np
import trueskill_through_time as ttt
import cProfile
from scipy.optimize import minimize_scalar


%load_ext autoreload
%autoreload 2

In [2]:
mdb_file = r"/mnt/c/Program Files (x86)/OnCourt/OnCourt.mdb"
password = "qKbE8lWacmYQsZ2"
atp = loader.TennisLoader(mdb_file, password)
atp.clean_games()

In [4]:
def prepare_games_data(df: pl.DataFrame) -> pl.DataFrame:
    # Convert Day column to month numbers for the rating calculation
    # We'll use the difference from the minimum date to ensure continuous months
    return (
        df.select(
            # Convert Day to continuous months from start
            (pl.col("Day") - pl.col("Day").min()).alias("Month"),
            # Keep player columns
            pl.col("P1").alias("White"),
            pl.col("P2").alias("Black"),
            # All games have score 1 for first player (White)
            pl.lit(1.0).alias("Score")
        )
        # Ensure proper types
        .with_columns([
            pl.col("Month").cast(pl.Int64),
            pl.col("White").cast(pl.Int64),
            pl.col("Black").cast(pl.Int64),
            pl.col("Score").cast(pl.Float64)
        ])
    )

# Optional: Create initial status for players who already have ratings
def create_initial_status(df: pl.DataFrame, init_rating: float = 2200) -> pl.DataFrame:
    # Get unique players
    all_players = pl.concat([
        df.select("P1").unique(),
        df.select("P2").unique()
    ]).unique()
    
    return pl.DataFrame({
        "Player": all_players.select("P1").to_series(),
        "Rating": [init_rating] * all_players.height,
        "Deviation": [300.0] * all_players.height,  # Initial deviation
        "Games": [0] * all_players.height,
        "Win": [0] * all_players.height,
        "Draw": [0] * all_players.height,
        "Loss": [0] * all_players.height,
        "Lag": [0] * all_players.height
    })


In [36]:
atp.games

P1,P2,Tour,Surface,Day,Date
i64,i64,i64,i8,i64,date
115,105,1,0,2568,1997-01-12
35,223,1,0,2558,1997-01-02
61,110,1,0,2558,1997-01-02
275,181,1,0,2558,1997-01-02
105,103,1,0,2558,1997-01-02
…,…,…,…,…,…
35036,25533,20505,1,12837,2025-02-23
66379,29905,20458,1,12837,2025-02-23
36617,26153,20327,1,12837,2025-02-23


In [59]:
result = stephenson(
    games=atp.games,
    init=(2200, 300),      # Default initial rating and deviation
    cval=10,               # Rating deviation increase parameter
    hval=10,               # Rating deviation decrease parameter
    bval=0,                # Score bias parameter
    lambda_=2,             # Learning rate parameter
    history=True           # Track rating history
)

Processing 967630 games across 426 months for 39250 players...
Processing month 0/426 (0.0%)
n_games dtype: int64
games_this_month dtype: float64
n_games first few values: [0 0 0 0 0]
games_this_month first few values: [0. 0. 0. 7. 0.]
n_games dtype: float64
games_this_month dtype: float64
n_games first few values: [0. 0. 0. 7. 0.]
games_this_month first few values: [0. 0. 7. 9. 0.]
n_games dtype: float64
games_this_month dtype: float64
n_games first few values: [ 0.  0.  7. 16.  0.]
games_this_month first few values: [ 0.  0. 11.  7.  0.]
n_games dtype: float64
games_this_month dtype: float64
n_games first few values: [ 0.  0. 18. 23.  0.]
games_this_month first few values: [0. 0. 0. 0. 0.]
n_games dtype: float64
games_this_month dtype: float64
n_games first few values: [ 0.  0. 18. 23.  0.]
games_this_month first few values: [0. 0. 3. 1. 0.]
n_games dtype: float64
games_this_month dtype: float64
n_games first few values: [ 0.  0. 21. 24.  0.]
games_this_month first few values: [0. 0.

In [60]:
print(result.ratings)

shape: (39_250, 8)
┌────────┬─────────────┬───────────┬───────┬─────┬──────┬──────┬─────┐
│ Player ┆ Rating      ┆ Deviation ┆ Games ┆ Win ┆ Draw ┆ Loss ┆ Lag │
│ ---    ┆ ---         ┆ ---       ┆ ---   ┆ --- ┆ ---  ┆ ---  ┆ --- │
│ i64    ┆ f64         ┆ f64       ┆ f64   ┆ i64 ┆ i64  ┆ i64  ┆ i64 │
╞════════╪═════════════╪═══════════╪═══════╪═════╪══════╪══════╪═════╡
│ 47275  ┆ 9206.935618 ┆ 350.0     ┆ 499.0 ┆ 363 ┆ 0    ┆ 136  ┆ 1   │
│ 29372  ┆ 9183.825951 ┆ 350.0     ┆ 794.0 ┆ 517 ┆ 0    ┆ 277  ┆ 0   │
│ 58591  ┆ 8809.268194 ┆ 350.0     ┆ 212.0 ┆ 133 ┆ 0    ┆ 79   ┆ 0   │
│ 65903  ┆ 8750.267901 ┆ 350.0     ┆ 302.0 ┆ 184 ┆ 0    ┆ 118  ┆ 0   │
│ 13674  ┆ 8539.549501 ┆ 350.0     ┆ 852.0 ┆ 492 ┆ 0    ┆ 360  ┆ 0   │
│ …      ┆ …           ┆ …         ┆ …     ┆ …   ┆ …    ┆ …    ┆ …   │
│ 73201  ┆ 342.851249  ┆ 350.0     ┆ 87.0  ┆ 2   ┆ 0    ┆ 85   ┆ 1   │
│ 3663   ┆ 295.671958  ┆ 350.0     ┆ 67.0  ┆ 2   ┆ 0    ┆ 65   ┆ 3   │
│ 11240  ┆ 257.569917  ┆ 350.0     ┆ 108.0 ┆ 3   ┆ 0    ┆ 

In [5]:
model_Elo = Elo(train_dataset.competitors)
model_Elo.fit_dataset(train_dataset)
test_probs_Elo = model_Elo.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics_Elo = binary_metrics_suite(probs=test_probs_Elo, outcomes=test_dataset.outcomes)
print(test_metrics_Elo)

{'accuracy': np.float64(0.704), 'accuracy_without_draws': np.float64(0.704), 'log_loss': np.float64(0.5663021558046971), 'brier_score': np.float64(0.19234192082790214)}


In [6]:
model_Glicko2 = Glicko2(train_dataset.competitors)
model_Glicko2.fit_dataset(train_dataset)
test_probs_Glicko2 = model_Glicko2.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics_Glicko2 = binary_metrics_suite(probs=test_probs_Glicko2, outcomes=test_dataset.outcomes)
print(test_metrics_Glicko2)

{'accuracy': np.float64(0.7194), 'accuracy_without_draws': np.float64(0.7194), 'log_loss': np.float64(0.5367293053714022), 'brier_score': np.float64(0.181082394663298)}


In [7]:
model_ttt = TrueSkillThroughTime(train_dataset, beta = 0.65)

In [8]:
test_probs_ttt = model_ttt.fit_dataset(test_dataset, return_pre_match_probs=True)

Iteration =  0 , step =  (15.562463294923685, 5.398507175164521)
Iteration =  1 , step =  (4.518934878496438, 1.4937610148366016)
Iteration =  2 , step =  (1.805928258507283, 0.6310129380403078)
Iteration =  3 , step =  (0.46799869370987857, 0.177791527890105)
Iteration =  4 , step =  (0.2397291069106212, 0.07503763530515384)
End
Iteration =  0 , step =  (4.72831637248748, 1.5935725538745946)
Iteration =  1 , step =  (0.13646043318573842, 0.1627428207876499)
Iteration =  2 , step =  (0.04735007100306454, 0.007391870626114239)
Iteration =  3 , step =  (0.03173535524211424, 0.0046691022294547135)
Iteration =  4 , step =  (0.02144279949695793, 0.0029871183188872052)
End
Iteration =  0 , step =  (1.2164705285884847, 0.6797268027501144)
Iteration =  1 , step =  (0.18120552248655075, 0.029175366160491656)
Iteration =  2 , step =  (0.019422506869788858, 0.002746304450390369)
Iteration =  3 , step =  (0.005744642699580638, 0.0007257715621609151)
End
Iteration =  0 , step =  (5.773846043467827,

KeyboardInterrupt: 

In [82]:
test_metrics_ttt = binary_metrics_suite(probs=test_probs_ttt, outcomes=test_dataset.outcomes)
print(test_metrics_ttt)

{'accuracy': np.float64(0.7216), 'accuracy_without_draws': np.float64(0.7216), 'log_loss': np.float64(0.543611409578523), 'brier_score': np.float64(0.18294739639909088)}


In [26]:
test_probs_ttt = model_ttt.fit_dataset(test_dataset)

TypeError: TrueSkillThroughTime.add_games() takes from 2 to 3 positional arguments but 5 were given

In [136]:
def objective_function_EloPlus(lambda_reg, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
    
    Parameters:
        gamma: The parameter to optimize
        dataset: Dataset containing competitors
        train_dataset: Training dataset
        test_dataset: Test dataset for evaluation
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = EloPlus(dataset.competitors, lambda_reg=lambda_reg, total_iterations = 50, update_method = 'batched')
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("lambda_reg of: ", lambda_reg, " gives brier of: ", brier)
    return test_metrics['brier_score']

In [ ]:
def objective_function_Elo(k, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = Elo(dataset.competitors, k=k)
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("K of: ", k, " gives brier of: ", brier)
    return brier

In [92]:
def objective_function_Glicko(c, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = Glicko(dataset.competitors, c = c)
    model.fit_dataset(train_dataset)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("c of: ", c, " gives brier of: ", brier)
    return brier

In [69]:
def objective_function_ttt(beta, dataset, train_dataset, test_dataset):
    """
    Objective function to minimize. Takes gamma and returns Brier score.
        
    Returns:
        float: Brier score (lower is better)
    """
    # Initialize model with current gamma
    model = TrueSkillThroughTime(train_dataset, beta = beta)
    model.iterate(5)
    test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True, iterations = 5)
    test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
    brier = test_metrics['brier_score']
    print("Beta of: ", beta, " gives brier of: ", brier)
    return brier

In [70]:
result = minimize_scalar(
    objective_function_ttt,
    args=(dataset, train_dataset, test_dataset),
    method='brent',  # Best general-purpose 1D optimizer
)

Iteration =  0 , step =  (17.0282089382114, 5.655630358950269)
Iteration =  1 , step =  (8.879721459636873, 2.209164802281448)
Iteration =  2 , step =  (6.121663292227514, 1.6556274525094823)
Iteration =  3 , step =  (2.468722529145867, 0.6672988785335886)
Iteration =  4 , step =  (1.1472482672578521, 0.3722250327117397)
End
Limited iteration: 0
Limited iteration: 1
Limited iteration: 2
Limited iteration: 3
Limited iteration: 4
Iteration =  0 , step =  (12.596027960104927, 5.026531001012914)
Iteration =  1 , step =  (6.0155176198913125, 3.161192659036378)
Iteration =  2 , step =  (1.2047535499509334, 0.5338058919625286)
Iteration =  3 , step =  (0.18828318648172016, 0.057710441215105046)
Iteration =  4 , step =  (0.056342955536241135, 0.019461579838512932)
End
Limited iteration: 0
Limited iteration: 1
Limited iteration: 2
Limited iteration: 3
Limited iteration: 4
Iteration =  0 , step =  (12.553509304455064, 4.3056275031837)
Iteration =  1 , step =  (1.8540022170034662, 1.4818921750831

KeyboardInterrupt: 

In [80]:
# Get results
print("Optimal variable", result.x, "gives brier of", result.fun)

Optimal variable 9.465461359819516 gives brier of 0.19587134446883256


In [9]:
a = atp.to_ttt_format(sample_games=50000)
h = ttt.History(composition = a[0], times = a[1])

Speedup branch


In [10]:
h.convergence(epsilon = 0.01, iterations = 10)

Iteration =  0 , step =  (14.945056624654391, 5.377917096145063)
Iteration =  1 , step =  (2.812648566510126, 0.9406496200515968)
Iteration =  2 , step =  (0.6736415422089026, 0.19307175884181205)
Iteration =  3 , step =  (0.16691655031202535, 0.050757047021259494)
Iteration =  4 , step =  (0.056196947812871656, 0.016280531610799898)
Iteration =  5 , step =  (0.02045164473815486, 0.005719036192219917)
Iteration =  6 , step =  (0.017492529891368847, 0.0033196865249522922)
Iteration =  7 , step =  (0.016703107565909647, 0.0030154666622732584)
Iteration =  8 , step =  (0.015375995091575945, 0.002739081857586978)
Iteration =  9 , step =  (0.014051774145483975, 0.0025132449104439125)
End


((0.014051774145483975, 0.0025132449104439125), 10)

In [28]:
dataset.matchups

array([[10617, 23969],
       [24225, 33677],
       [13428,  4026],
       ...,
       [34224, 36744],
       [30320, 16045],
       [18871, 34219]], dtype=int32)

P1,P2,Tour,Surface,Day,Date
i64,i64,i64,i8,i64,date
115,105,1,0,2568,1997-01-12
35,223,1,0,2558,1997-01-02
61,110,1,0,2558,1997-01-02
275,181,1,0,2558,1997-01-02
105,103,1,0,2558,1997-01-02
…,…,…,…,…,…
88766,44555,20203,0,12739,2024-11-17
50504,99846,20211,0,12738,2024-11-16
99982,84556,20211,0,12738,2024-11-16


In [114]:
mu = [tp[1].mu for tp in h.learning_curves()[5992]]

In [138]:
players = [5992, 22807, 30470, 19, 47275]

[h.learning_curves()[player][-1] for player in players]

[(12704, N(mu=7.452, sigma=0.439)),
 (12736, N(mu=6.523, sigma=0.400)),
 (12723, N(mu=6.236, sigma=0.375)),
 (11510, N(mu=6.346, sigma=0.489)),
 (12739, N(mu=8.172, sigma=0.434))]

In [120]:
whr = whole_history_rating.Base()
whr.load_games(atp.to_whr_format())

In [135]:
whr.iterate(5)

In [50]:
import time
import numpy as np
from dataclasses import dataclass

# Current approach with Gaussian objects
class Gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    def __mul__(self, other):
        pi1, pi2 = self.sigma**-2, other.sigma**-2
        pi = pi1 + pi2
        tau = (self.mu * pi1 + other.mu * pi2) / pi
        sigma = np.sqrt(1/pi)
        return Gaussian(tau, sigma)

# New approach using just numbers
@dataclass
class FastMessage:
    mu: float
    sigma: float

def multiply_messages(m1: FastMessage, m2: FastMessage) -> FastMessage:
    pi1, pi2 = m1.sigma**-2, m2.sigma**-2
    pi = pi1 + pi2
    tau = (m1.mu * pi1 + m2.mu * pi2) / pi
    sigma = np.sqrt(1/pi)
    return FastMessage(tau, sigma)

def benchmark(n_messages=10000, n_iterations=5):
    """
    Benchmark comparing Gaussian objects vs direct numerical operations.
    
    Args:
        n_messages: Number of message updates to perform
        n_iterations: Number of iterations through the messages
    """
    # Generate random data
    np.random.seed(42)
    means = np.random.randn(n_messages)
    stds = np.abs(np.random.randn(n_messages)) + 1
    
    # Test Gaussian objects
    start = time.time()
    messages = [Gaussian(mu, sigma) for mu, sigma in zip(means, stds)]
    for _ in range(n_iterations):
        for i in range(1, len(messages)):
            messages[i] = messages[i] * messages[i-1]
    gaussian_time = time.time() - start
    
    # Test direct numerical
    start = time.time()
    fast_messages = [FastMessage(mu, sigma) for mu, sigma in zip(means, stds)]
    for _ in range(n_iterations):
        for i in range(1, len(fast_messages)):
            fast_messages[i] = multiply_messages(fast_messages[i], fast_messages[i-1])
    numerical_time = time.time() - start
    
    return {
        'gaussian_time': gaussian_time,
        'numerical_time': numerical_time,
        'speedup': gaussian_time / numerical_time
    }

# Run benchmarks with different sizes
sizes = [1000, 10000, 100000]
for size in sizes:
    print(f"\nBenchmark with {size} messages:")
    results = benchmark(n_messages=size)
    print(f"Gaussian objects: {results['gaussian_time']:.3f} seconds")
    print(f"Direct numerical: {results['numerical_time']:.3f} seconds")
    print(f"Speedup factor: {results['speedup']:.2f}x")


Benchmark with 1000 messages:
Gaussian objects: 0.024 seconds
Direct numerical: 0.008 seconds
Speedup factor: 2.99x

Benchmark with 10000 messages:
Gaussian objects: 0.097 seconds
Direct numerical: 0.152 seconds
Speedup factor: 0.64x

Benchmark with 100000 messages:
Gaussian objects: 0.957 seconds
Direct numerical: 1.899 seconds
Speedup factor: 0.50x
